In [28]:
import os
import string
import numpy as np
import pandas as pd
import re
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from string import digits
import logging
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
import time
logging.getLogger('tensorflow').setLevel(logging.FATAL)
import matplotlib.ticker as ticker
import unicodedata
import io
import warnings
import sys

In [92]:
for dirname, _, filenames in os.walk('/ISL_Project/ISL_Phase_2'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

PATH = "../ISL_Phase_2/islcorpus.csv"

In [93]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

#def preprocess_sentence(w):
#    w = unicode_to_ascii(w.lower().strip())
#    w = re.sub(r"([?.!,¿])", r" \1 ", w)
#    w = re.sub(r'[" "]+', " ", w)
#    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
#    w = w.rstrip().strip()
#    return w

In [115]:
def preprocess_input(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    return w

def preprocess_output(w):
    w = w.rstrip().strip()
    return w

In [116]:
def create_dataset(path = PATH):
    lines = pd.read_csv(path,encoding ='utf-8')
    lines = lines.dropna()
    ip = []
    op = []
    for i,j in zip(lines['Input'],lines['Output']):
        ip_1 = [preprocess_input(w) for w in i.split(' ')]
        ip_1.append('<end>')
        ip_1.insert(0,'<start>')
        op_1 = [preprocess_output(w) for w in j.split(' ')]
        op_1.append('<end>')
        op_1.insert(0,'<start>')
        ip.append(ip_1)
        op.append(op_1)
    return op,ip

In [117]:
def max_length(tensor):
    return max(len(t) for t in tensor)

### Tokenization

In [118]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = '')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
    print(tensor,lang_tokenizer)
    return tensor, lang_tokenizer

In [119]:
def load_dataset(path = PATH):
    targ_lang,inp_lang = create_dataset(path)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor,inp_lang_tokenizer, targ_lang_tokenizer

In [120]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(PATH)
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)
print(input_tensor, target_tensor, inp_lang, targ_lang,max_length_targ, max_length_inp )

[[  1   8  15  16  17  18  19   2]
 [  1  34   7  15   5   2   0   0]
 [  1   4  20  10   2   0   0   0]
 [  1   4  69   5   2   0   0   0]
 [  1   9  25  26  35   2   0   0]
 [  1  12  36  27   2   0   0   0]
 [  1   8  28  37  38   2   0   0]
 [  1   4  70  39   5   2   0   0]
 [  1  71  27   4   2   0   0   0]
 [  1   4  21   2   0   0   0   0]
 [  1  72  73  74   2   0   0   0]
 [  1   4  11   2   0   0   0   0]
 [  1  11  75   2   0   0   0   0]
 [  1  76  40   2   0   0   0   0]
 [  1  77  78   2   0   0   0   0]
 [  1  79  80   2   0   0   0   0]
 [  1  81  82   2   0   0   0   0]
 [  1  83  84   2   0   0   0   0]
 [  1   4  41   2   0   0   0   0]
 [  1  85  42  29   2   0   0   0]
 [  1  11  86   2   0   0   0   0]
 [  1   7  87   5   2   0   0   0]
 [  1  88   2   0   0   0   0   0]
 [  1   3  89   4   2   0   0   0]
 [  1  11  90   2   0   0   0   0]
 [  1   4  43   2   0   0   0   0]
 [  1  11  91   2   0   0   0   0]
 [  1   4  44   2   0   0   0   0]
 [  1   4  92   2   

### Create Train and Test Dataset

In [121]:
input_tensor_train, input_tensor_val, target_tensor_train,target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

111 111 13 13


In [122]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print("%d -------  %s" % (t, lang.index_word[t]))


In [123]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 -------  <start>
3 -------  i
152 -------  three
66 -------  ball
2 -------  <end>

Target Language; index to word mapping
1 -------  <start>
3 -------  i
40 -------  have
188 -------  three
189 -------  balls
2 -------  <end>


### Create Dataset

In [136]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 128
units = 256
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [153]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [154]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

In [168]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [169]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits = True, reduction = 'none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real,pred)
    mask = tf.cast(mask, dtype = loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [170]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [171]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
    # Teacher forcing
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))      
    return batch_loss

In [173]:
EPOCHS = 20

for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.0951
Epoch 1 Loss 2.0951
Time taken for 1 epoch 0.13541126251220703 sec

Epoch 2 Batch 0 Loss 2.0857
Epoch 2 Loss 2.0857
Time taken for 1 epoch 0.26532983779907227 sec

Epoch 3 Batch 0 Loss 2.0815
Epoch 3 Loss 2.0815
Time taken for 1 epoch 0.12466740608215332 sec

Epoch 4 Batch 0 Loss 2.0196
Epoch 4 Loss 2.0196
Time taken for 1 epoch 0.2056117057800293 sec

Epoch 5 Batch 0 Loss 2.0544
Epoch 5 Loss 2.0544
Time taken for 1 epoch 0.12901592254638672 sec

Epoch 6 Batch 0 Loss 1.9756
Epoch 6 Loss 1.9756
Time taken for 1 epoch 0.24234914779663086 sec

Epoch 7 Batch 0 Loss 1.9377
Epoch 7 Loss 1.9377
Time taken for 1 epoch 0.13364362716674805 sec

Epoch 8 Batch 0 Loss 2.0878
Epoch 8 Loss 2.0878
Time taken for 1 epoch 0.2641305923461914 sec

Epoch 9 Batch 0 Loss 2.0263
Epoch 9 Loss 2.0263
Time taken for 1 epoch 0.15060114860534668 sec

Epoch 10 Batch 0 Loss 1.9562
Epoch 10 Loss 1.9562
Time taken for 1 epoch 0.22066020965576172 sec

Epoch 11 Batch 0 Loss 2.0558
Epoch 11 Lo

In [174]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = preprocess_input(sentence)
    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '
        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [175]:
def translate(sentence):
    result, sentence = evaluate(sentence)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [176]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [177]:
translate(u'tomorrow together we go')

Input: tomorrow together we go
Predicted translation: <end> 
